In [1]:
# Remember: library imports are ALWAYS at the top of the script, no exceptions!
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

# for better resolution plots
%config InlineBackend.figure_format = 'retina' # optionally, you can change 'svg' to 'retina'

# Seeting seaborn style
sns.set()

In [185]:
df_train = pd.read_excel(r'Train.xlsx')

In [186]:
df_train.head()

,CITIZEN_ID,Name,Birthday,Native Continent,Marital Status,Lives with,Base Area,Education Level,Years of Education,Employment Sector,Role,Working Hours per week,Money Received,Ticket Price,Income
0,12486,Mr. Adam Glover,"July 1,2003",Europe,Married,Wife,Northbury,High School + PostGraduation,13,Private Sector - Services,Repair & constructions,40,0,2273,1
1,12487,Mr. Cameron McDonald,"January 25,2006",Europe,Married,Wife,Northbury,Professional School,12,Public Sector - Others,Repair & constructions,40,0,0,1
2,12488,Mr. Keith Davidson,"May 10,2009",Europe,Married,Wife,Northbury,Professional School,12,Private Sector - Services,Sales,46,0,2321,1
3,12489,Mr. Alexander Gill,"March 25,1985",Europe,Married,Wife,Northbury,High School - 2nd Cycle,11,Private Sector - Services,Security,37,5395,0,1
4,12490,Mr. Neil Piper,"May 29,2015",Europe,Single,Other Family,Northbury,PhD,21,Self-Employed (Individual),Professor,45,0,0,1


In [7]:
df_test = pd.read_excel(r'Test.xlsx')

In [8]:
df_test.head()

,CITIZEN_ID,Name,Birthday,Native Continent,Marital Status,Lives with,Base Area,Education Level,Years of Education,Employment Sector,Role,Working Hours per week,Money Received,Ticket Price
0,34886,Mr. Dan Kerr,"May 13,2012",Europe,Married,Wife,Northbury,Bachelors + PostGraduation,16,Private Sector - Services,Sales,40,0,0
1,34887,Miss Olivia Grant,"August 9,2014",Europe,Single,Other Family,Northbury,Bachelors + PostGraduation,16,Private Sector - Services,Professor,36,0,0
2,34888,Mr. Trevor Simpson,"May 23,2020",Europe,Divorced,Other Family,Northbury,Professional School + PostGraduation,13,Private Sector - Services,Management,50,12940,0
3,34889,Mr. David Clark,"April 15,2016",Europe,Married,Wife,Northbury,Professional School,12,Private Sector - Services,Professor,48,0,1827
4,34890,Mr. Gavin Lambert,"April 22,2011",Europe,Married,Wife,Watford,Bachelors,15,Public Sector - Others,Professor,45,0,0


In [32]:
df_train["Base Area"].unique()

array(['Northbury', 'Auchenshuggle', 'Sharnwick', 'Laewaes',
       'MillerVille', '?', 'Fanfoss', 'Wigston', "Knife's Edge",
       'Aerilon', 'Watford', 'Lanercost', 'Bellmoral', 'Marnmouth',
       'Bellenau', 'Butterpond', 'Laenteglos', 'Tranmere', 'Eelry',
       'Alverton', 'Pran', 'Willesden', "King's Watch", 'Kald',
       'Kirkwall', "Fool's March", 'Middlesbrough', 'Orilon',
       'Aroonshire', 'Aberuthven', 'Sharpton', 'Lewes', 'Mensfield',
       'Cherrytown', 'Redwick Bush', 'Drumchapel', 'Carlisle',
       'Conriston', 'Woodpine', 'Ironforge'], dtype=object)

In [51]:
df_train[["Role","Base Area"]][df_train["Role"].isnull()]

,Role,Base Area


In [75]:
df_train[["Employment Sector", "Role"]][df_train["Employment Sector"]==df_train["Role"]]

,Employment Sector,Role
22,?,?
25,?,?
37,?,?
43,?,?
44,?,?
...,...,...
22309,?,?
22328,?,?
22337,?,?
22339,?,?


In [80]:
df_train["Base Area"][df_train["Base Area"]=="?"]

30       ?
97       ?
214      ?
285      ?
359      ?
        ..
22047    ?
22144    ?
22242    ?
22349    ?
22373    ?
Name: Base Area, Length: 395, dtype: object

In [81]:
df_train["Lives with"].unique()

array(['Wife', 'Other Family', 'Children', 'Other relatives', 'Alone',
       'Husband'], dtype=object)

In [86]:
df_train["Years of Education"].unique()

array([13, 12, 11, 21, 15, 16, 17, 10, 18,  8,  9,  6,  4,  2])

In [93]:
df_train.groupby("Education Level")["Years of Education"].mean()

Education Level
Bachelors                               15
Bachelors + PostGraduation              16
High School + PostGraduation            13
High School - 1st Cycle                 10
High School - 2nd Cycle                 11
High School Complete                    12
Masters                                 17
Masters + PostGraduation                18
Middle School - 1st Cycle                6
Middle School - 2nd Cycle                8
Middle School Complete                   9
PhD                                     21
Preschool                                2
Primary School                           4
Professional School                     12
Professional School + PostGraduation    13
Name: Years of Education, dtype: int64

In [99]:
df_train.groupby(["Education Level","Years of Education"] )["Years of Education"].mean()

Education Level                       Years of Education
Bachelors                             15                    15
Bachelors + PostGraduation            16                    16
High School + PostGraduation          13                    13
High School - 1st Cycle               10                    10
High School - 2nd Cycle               11                    11
High School Complete                  12                    12
Masters                               17                    17
Masters + PostGraduation              18                    18
Middle School - 1st Cycle             6                      6
Middle School - 2nd Cycle             8                      8
Middle School Complete                9                      9
PhD                                   21                    21
Preschool                             2                      2
Primary School                        4                      4
Professional School                   12                    1

In [102]:
df_train["Working Hours per week"].unique()

array([40, 46, 37, 45, 30, 20, 15, 50, 60, 54, 35, 49, 75, 55, 84, 25, 24,
       48, 21, 10,  8, 38, 32,  4, 16, 70, 26, 42, 90, 18, 28, 22, 36,  7,
       14, 29, 66, 58, 65, 80, 44, 12, 72, 47, 27, 17, 99, 81, 56,  3, 52,
       43,  5, 41, 57, 63,  9,  6, 34, 98, 19, 68, 62, 51, 91, 13, 33, 67,
       39,  1, 11, 23, 88, 96, 64, 53,  2, 31, 94, 78, 92, 74, 85, 95, 73,
       89, 76, 82, 87, 86, 59, 97, 77, 61])

In [114]:
df_train["Working Hours per week"][df_train["Working Hours per week"]<=5]

130      4
746      3
826      5
1216     5
1392     5
        ..
21257    1
21347    5
21764    1
22165    5
22359    2
Name: Working Hours per week, Length: 140, dtype: int64

In [126]:
df_train["Ticket Price"][df_train["Ticket Price"]!=0]

0        2273
2        2321
41       2970
67       2432
87       2462
         ... 
22327    2339
22329    2432
22359    2234
22373    2321
22390    2339
Name: Ticket Price, Length: 1057, dtype: int64

In [128]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22400 entries, 0 to 22399
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CITIZEN_ID              22400 non-null  int64 
 1   Name                    22400 non-null  object
 2   Birthday                22400 non-null  object
 3   Native Continent        22400 non-null  object
 4   Marital Status          22400 non-null  object
 5   Lives with              22400 non-null  object
 6   Base Area               22400 non-null  object
 7   Education Level         22400 non-null  object
 8   Years of Education      22400 non-null  int64 
 9   Employment Sector       22400 non-null  object
 10  Role                    22400 non-null  object
 11  Working Hours per week  22400 non-null  int64 
 12  Money Received          22400 non-null  int64 
 13  Ticket Price            22400 non-null  int64 
 14  Income                  22400 non-null  int64 
dtypes:

In [130]:
df_train.describe(include="all")

,CITIZEN_ID,Name,Birthday,Native Continent,Marital Status,Lives with,Base Area,Education Level,Years of Education,Employment Sector,Role,Working Hours per week,Money Received,Ticket Price,Income
count,22400.000000,22400,22400,22400,22400,22400,22400,22400,22400.000000,22400,22400,22400.000000,22400.000000,22400.000000,22400.000000
unique,NaN,16074,11257,5,7,6,40,16,NaN,9,15,NaN,NaN,NaN,NaN
top,NaN,Mr. Keith Mills,"August 15,2025",Europe,Married,Wife,Northbury,Professional School,NaN,Private Sector - Services,Professor,NaN,NaN,NaN,NaN
freq,NaN,4,9,19111,10219,9012,20074,7232,NaN,15599,2849,NaN,NaN,NaN,NaN
mean,23685.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.173884,NaN,NaN,40.483795,1324.915357,109.145313,0.237098
std,6466.467351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.512451,NaN,NaN,12.370921,9227.771813,500.208904,0.425313
min,12486.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,1.000000,0.000000,0.000000,0.000000
25%,18085.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000000,NaN,NaN,40.000000,0.000000,0.000000,0.000000
50%,23685.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.000000,NaN,NaN,40.000000,0.000000,0.000000,0.000000
75%,29285.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.000000,NaN,NaN,45.000000,0.000000,0.000000,0.000000


In [187]:
#FUNCIONA
df_train.set_index("CITIZEN_ID", inplace=True)

In [208]:
#FUNCIONA
from datetime import datetime

df_train['Birthday']=df_train['Birthday'].map(lambda x: x.replace("February 29", "February 28"))

In [210]:
#funciona
df_train['Birthday'][df_train['Birthday'].str.contains("February 29")]

Series([], Name: Birthday, dtype: object)

In [211]:
#funciona
df_train['Birthday']=df_train['Birthday'].map(lambda x: datetime.strptime(x, " %B %d,%Y").date())

In [212]:
#funciona
df_train['Birthday']

CITIZEN_ID
12486    2003-07-01
12487    2006-01-25
12488    2009-05-10
12489    1985-03-25
12490    2015-05-29
            ...    
34881    2023-03-15
34882    2025-03-07
34883    2022-07-20
34884    2011-06-11
34885    2026-10-02
Name: Birthday, Length: 22400, dtype: object

In [215]:
#funciona
df_train['Birthday'] = pd.to_datetime(df_train['Birthday'])